In [ ]:
# 安装需要的项目和包
!git clone https://github.com/huggingface/transformers.git
!pip install ./transformers
!pip install sentencepiece


In [ ]:
# 加载模型
# 如果加载ClueAI/PromptCLUE-base-v1-5, 请替换为ClueAI/PromptCLUE-base-v1-5
from transformers import T5Tokenizer, T5ForConditionalGeneration
tokenizer = T5Tokenizer.from_pretrained("ClueAI/PromptCLUE-base")
model = T5ForConditionalGeneration.from_pretrained("ClueAI/PromptCLUE-base")

In [ ]:
# 使用
import torch
from transformers import AutoTokenizer
# 修改colab笔记本设置为gpu，推理更快
device = torch.device('cuda')
model.to(device)
def preprocess(text):
  return text.replace("\n", "_")

def postprocess(text):
  return text.replace("_", "\n")

def answer(text, sample=True, top_p=0.6):
  '''sample：是否抽样。生成任务，可以设置为True;
  top_p：0-1之间，生成的内容越多样'''
  text = preprocess(text)
  encoding = tokenizer(text=[text], truncation=True, padding=True, max_length=768, return_tensors="pt").to(device)
  if not sample:
    out = model.generate(**encoding, return_dict_in_generate=True, output_scores=False, max_new_tokens=128, num_beams=4, length_penalty=0.6)
  else:
    out = model.generate(**encoding, return_dict_in_generate=True, output_scores=False, max_new_tokens=128, do_sample=True, top_p=top_p)
  out_text = tokenizer.batch_decode(out["sequences"], skip_special_tokens=True)
  return postprocess(out_text[0])
print("end...")

In [ ]:
#读取数据
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/
import json
import csv
import pandas as pd
entities = ['character', 'country', 'material', 'weapon', 'element', 'weapontype']
results = dict()
for entity in entities:
    file = open({entity}.json, encoding='utf-8')
    result = {}
    count = 0
    for line in file:
        line = line.strip()  # 去掉首尾空白字符
        if line:
            try:
                item_dict = json.loads(line)
                result[count]=item_dict
                count+=1
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e} - Line: {line}")
    results[entity]=result

file = open(character.json, encoding='utf-8')
result = {}
count = 0
for line in file:
    line = line.strip()  # 去掉首尾空白字符
    if line:
        try:
            item_dict = json.loads(line)
            result[count]=item_dict
            count+=1
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e} - Line: {line}")


headers = ['END_ID', 'type', 'START_ID']
with open('character2character.csv', 'w', encoding='utf-8-sig', newline="") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(headers)
    for chId in result.keys():
        startId = '#'+str(chId)
        cname = result[chId]['所在国家']
        for cId in result.keys():
            if result[cId]['国家名称'] == cname:
                data = [['c'+str(cId), '来自', startId]]
                #print(data)
                csv_writer.writerows(data)